<a href="https://colab.research.google.com/github/thangdepzai/DL-in-Pytorch/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [3]:
from torchvision import datasets 

In [4]:
from torchvision import transforms

In [5]:
import matplotlib.pyplot as plt

In [6]:
data_path = 'https://drive.google.com/drive/folders/1VbY84GVkCY0wg6bu7JpsBGtFJeDXsGp9?usp=sharing/'

In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)

cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)


Extracting https://drive.google.com/drive/folders/1VbY84GVkCY0wg6bu7JpsBGtFJeDXsGp9?usp=sharing/cifar-10-python.tar.gz to https://drive.google.com/drive/folders/1VbY84GVkCY0wg6bu7JpsBGtFJeDXsGp9?usp=sharing/
Files already downloaded and verified


In [8]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [9]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [21]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=50,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=50,
                                           shuffle=True)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
print(device)

cuda:0


In [15]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=3e-2)
loss_fn = nn.CrossEntropyLoss()
model.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [16]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()     
            loss.backward() 
            optimizer.step()
            loss_train += loss.item()
        correct = 0
        with torch.no_grad():
            for data in val_loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                c = (predicted == labels).squeeze()
                correct += c.sum()
        if epoch == 1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}, Val accuracy {}'.format(
                epoch,
                loss_train / len(train_loader),
                correct / len(cifar10_val)))

In [22]:
training_loop(
    n_epochs = 30,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    val_loader = val_loader
)

Epoch 1, Training loss 0.6749326125234365, Val accuracy 0.5543000102043152
Epoch 2, Training loss 0.4322879840731621, Val accuracy 0.5900999903678894
Epoch 3, Training loss 0.37380447129905225, Val accuracy 0.5834000110626221
Epoch 4, Training loss 0.36036988551542165, Val accuracy 0.5809999704360962
Epoch 5, Training loss 0.34760158028453586, Val accuracy 0.5898000001907349
Epoch 6, Training loss 0.3269571759328246, Val accuracy 0.5874999761581421
Epoch 7, Training loss 0.3197372918576002, Val accuracy 0.5845999717712402
Epoch 8, Training loss 0.32864086207933724, Val accuracy 0.5863999724388123
Epoch 9, Training loss 0.31216120410710574, Val accuracy 0.5861999988555908
Epoch 10, Training loss 0.3013608901798725, Val accuracy 0.5819000005722046
Epoch 11, Training loss 0.3053319010622799, Val accuracy 0.5910999774932861
Epoch 12, Training loss 0.30171274847909807, Val accuracy 0.5809000134468079
Epoch 13, Training loss 0.29969460604712367, Val accuracy 0.5967999696731567
Epoch 14, Trai